<a href="https://colab.research.google.com/github/KennyThinh/pytorch/blob/main/02_BaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as f #function without parameter relu, tank
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms #transformation to perform on dataset

In [ ]:
# create fully connected network
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)
  
  def forward(self, x):
    x = f.relu(self.fc1(x))
    x = self.fc2(x)
    return x

#test
model = NN(768, 10)
x = torch.rand(64,768)
print(model(x).shape)

torch.Size([64, 10])


In [ ]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
# hyper parameter
input_size = 784
num_classes = 10
num_epochs = 1
batch_size = 64
learning_rate = 0.001

In [51]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz -P dataset/
!tar -zxvf dataset/MNIST.tar.gz -C dataset/

train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root="dataset/", train = False, transform=transforms.ToTensor(), download=False)
test_loader = DataLoader(test_dataset, batch_size= batch_size, shuffle = True)

--2021-03-19 02:27:33--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-19 02:27:33--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘dataset/MNIST.tar.gz’

MNIST.tar.gz            [      <=>           ]  33.20M  30.5MB/s    in 1.1s    

2021-03-19 02:27:34 (30.5 MB/s) - ‘dataset/MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-uby

In [60]:
#Init the network
model = NN(input_size, num_classes).to(device)

In [61]:
#loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [63]:
#train 
for epoch in range(num_epochs):
  for batch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)
    
    data = data.reshape(data.shape[0], -1)        
    
    #forward
    scores = model(data)
    loss = criterion(scores, target)

    #backward
    optimizer.zero_grad() #to clear out gradients of last time
    loss.backward()

    #gradient to update params
    optimizer.step() 

In [81]:
#check acc on train and test
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval() #inform that we are going to evaluate
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device)
      y = y.to(device)

      x = x.reshape(x.shape[0], -1)
      scores = model(x) #dim (64,10)      
      _, predictions = torch.max(scores, dim=1) #find the max among 10 outputs, so the dim is 1
      #predictions ([64])
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0) #each time is 64 examples
    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)


Got 57158/60000 with accuracy 95.26
Got 9489/10000 with accuracy 94.89
